In [1]:
%pip install vosk
%pip install pyaudio
%pip install transformers
%pip install wave
%pip install googletrans==4.0.0-rc1
%pip install pyttsx3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [2]:
from googletrans import Translator
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
import json
import wave
import sys
from vosk import Model, KaldiRecognizer
import pyaudio
import re
import pyttsx3
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [152]:
# Step 0: Records Audio Sample

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 3

with wave.open('human_input.wav', 'wb') as wf:
    p = pyaudio.PyAudio()
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)

    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True)

    # print('Recording...')
    # for _ in range(0, RATE // CHUNK * RECORD_SECONDS):
    #     wf.writeframes(stream.read(CHUNK))
    # print('Done')

    print('Recording...')
    try:
        while True:
            wf.writeframes(stream.read(CHUNK))
    except KeyboardInterrupt:
        pass
    
    print('Done')

    stream.close()
    p.terminate()


Recording...
Done


In [12]:
# Play Audio Section

if len(sys.argv) < 2:
    print(f'Plays a wave file. Usage: {sys.argv[0]} filename.wav')
    sys.exit(-1)

with wave.open("human_input.wav", 'rb') as wf:
    # Instantiate PyAudio and initialize PortAudio system resources (1)
    p = pyaudio.PyAudio()

    # Open stream (2)
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    # Play samples from the wave file (3)
    while len(data := wf.readframes(CHUNK)):  # Requires Python 3.8+ for :=
        stream.write(data)
        
    # Close stream (4)
    stream.close()

    # Release PortAudio system resources (5)
    p.terminate()

In [153]:
# Step 1: Chinese Speech Recognition with Vosk

# Path to the downloaded Vosk model
model_path = "vosk-model-small-cn-0.22"

# Path to the input WAV file
wav_file_path = "human_input.wav"

# Load the Vosk model
model = Model(model_path)

# Open the WAV file
with wave.open(wav_file_path, "rb") as wf:
    # Check if the audio file has the correct parameters
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() not in [8000, 16000, 32000, 44100, 48000]:
        # print("Audio file must be WAV format mono PCM.")
        raise Exception("Audio file must be WAV format mono PCM.")

    # Create a Kaldi recognizer with the model and the sample rate
    recognizer = KaldiRecognizer(model, wf.getframerate())

    # Read the audio data and transcribe it
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if recognizer.AcceptWaveform(data):
            result = recognizer.Result()
            # print(result)
            text = json.loads(result).get('text', '')
            # print(f"Recognized text: {text}")

    # Final result
    final_result = recognizer.FinalResult()
    final_result_test = json.loads(final_result).get('text', '')

    if final_result_test != "":
        chinese_input_text = json.loads(final_result).get('text', '')
        chinese_input_text = re.sub(r"\s+", "", chinese_input_text, flags=re.UNICODE)
    else:
        chinese_input_text = re.sub(r"\s+", "", text, flags=re.UNICODE)

    

    print(f"Final recognized text: {chinese_input_text}")

Final recognized text: 服务员我要饺子


In [154]:
# Step 2: Translate from Chinese to English 


# Initialize the translator
translator = Translator()

# Translate from Chinese to English
translated = translator.translate(chinese_input_text, src='zh-cn', dest='en')
print(translated.text)  



Waiter I want dumplings


In [161]:
# Step 3

# Load the pre-trained model and tokenizer
# model_name = "microsoft/DialoGPT-small"
model_name = "microsoft/DialoGPT-medium"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_size="left")

# chatting 5 times with Top K sampling & tweaking temperature
for step in range(1):
    # take user input
    # text = input(">> You:")
    text = translated.text
    print(text)
    # encode the input and add end of string token
    input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors="pt")
    # concatenate new user input with chat history (if there is)
    bot_input_ids = torch.cat([chat_history_ids, input_ids], dim=-1) if step > 0 else input_ids
    # generate a bot response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        do_sample=True,
        top_k=100,
        temperature=0.63,
        pad_token_id=tokenizer.eos_token_id
    )
    #print the output
    output_text = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    print(f"DialoGPT: {output_text}")

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Waiter I want dumplings
DialoGPT: I want waffles


In [162]:
# Step 4:  Translate back to Chinese

# Translate from English to Chinese
translated_to_zh = translator.translate(output_text, src='en', dest='zh-cn')
print(translated_to_zh.text)  

我想要华夫饼


In [163]:
# Step 5: Translate reply back to Chinese 

# Initialize the TTS engine
engine = pyttsx3.init()

# Set properties before adding anything to speak
engine.setProperty('rate', 125)  # Speed percent (can go over 100)
engine.setProperty('volume', 1)  # Volume 0-1

# List available voices and set the voice to a Chinese one if available
voices = engine.getProperty('voices')
for voice in voices:
    # print(f"Voice: {voice.name}, ID: {voice.id}, Languages: {voice.languages}")
    if 'ZH' in voice.languages or 'ZH-CN' in voice.id:
        engine.setProperty('voice', voice.id)
        break

# Text to be spoken
text_to_speak = translated_to_zh.text

# Speak the text
engine.say(text_to_speak)
engine.runAndWait()